In [1]:
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping, ModelCheckpoint

# 경로 참 더럽네
from CRNNKerasmaster.parameter import *
from CRNNKerasmaster.Model import get_Model
from CRNNKerasmaster.Image_Generator import TextImageGenerator

import cv2
import itertools, os, time
import numpy as np
import argparse
import easydict
from datetime import datetime

K.set_learning_phase(0)

2022-12-19 14:47:34.628536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 14:47:34.816969: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 14:47:34.817018: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-19 14:47:35.579961: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
model = get_Model(training=True)

2022-12-19 14:47:36.720052: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ghwns82/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-12-19 14:47:36.720112: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-19 14:47:36.720182: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-HN4KDSG): /proc/driver/nvidia/version does not exist
2022-12-19 14:47:36.720498: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate 

In [4]:
try:
    model.load_weights('LSTM+BN4--26--0.011.hdf5')
    print("...Previous weight data...")
except:
    print("...New weight data...")
    pass

...New weight data...


In [5]:
train_file_path = './CRNNKerasmaster/DB/train/'
tiger_train = TextImageGenerator(train_file_path, img_w, img_h, batch_size, downsample_factor)
tiger_train.build_data()

9  Image Loading start...
True
9  Image Loading finish...


In [6]:
valid_file_path = './CRNNKerasmaster/DB/test/'
tiger_val = TextImageGenerator(valid_file_path, img_w, img_h, val_batch_size, downsample_factor)
tiger_val.build_data()

5  Image Loading start...
True
5  Image Loading finish...


In [7]:
# tf.config.run_functions_eagerly(True)

In [8]:
time_str='%Y_%m_%d_%H_%M_%S'

ada = Adadelta()

early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=4, mode='min', verbose=1)
checkpoint = ModelCheckpoint(filepath=f'LSTM+BN5--{datetime.now().strftime(time_str)}.hdf5', monitor='loss', verbose=10, mode='min', period=100)
# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=ada)

# captures output of softmax so we can decode the output during visualization
model.fit_generator(generator=tiger_train.next_batch(),
                    steps_per_epoch=int(tiger_train.n / batch_size),
                    epochs=1200,
                    callbacks=[checkpoint],
                    validation_data=tiger_val.next_batch(),
                    validation_steps=int(tiger_val.n / val_batch_size))

Epoch 1/1200
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1/1 [==============================] - 6s 6s/step - loss: 91.0585 - val_loss: 95.1287
Epoch 2/1200
1/1 [==============================] - 1s 517ms/step - loss: 84.4175 - val_loss: 95.5598
Epoch 3/1200
1/1 [==============================] - 1s 512ms/step - loss: 87.4812 - val_loss: 93.7206
Epoch 4/1200
1/1 [==============================] - 0s 465ms/step - loss: 92.2587 - val_loss: 93.5795
Epoch 5/1200
1/1 [==============================] - 0s 482ms/step - loss: 87.7221 - val_loss: 94.8889
Epoch 6/1200
1/1 [==============================] - 0s 451ms/step - loss: 90.7402 - val_loss: 94.6604
Epoch 7/1200
1/1 [==============================] - 0s 457ms/step - loss: 79.5205 - val_loss: 95.0010
Epoch 8/1200
1/1 [==============================] - 0s 474ms/step - loss: 83.5058 - val_

Epoch 75/1200
1/1 [==============================] - 0s 395ms/step - loss: 41.6031 - val_loss: 92.2063
Epoch 76/1200
1/1 [==============================] - 0s 419ms/step - loss: 39.9811 - val_loss: 95.1470
Epoch 77/1200
1/1 [==============================] - 0s 453ms/step - loss: 41.6445 - val_loss: 92.5651
Epoch 78/1200
1/1 [==============================] - 0s 394ms/step - loss: 47.9467 - val_loss: 88.8161
Epoch 79/1200
1/1 [==============================] - 0s 405ms/step - loss: 68.8000 - val_loss: 91.2300
Epoch 80/1200
1/1 [==============================] - 0s 437ms/step - loss: 53.3389 - val_loss: 91.5398
Epoch 81/1200
1/1 [==============================] - 0s 393ms/step - loss: 44.6957 - val_loss: 91.4620
Epoch 82/1200
1/1 [==============================] - 0s 417ms/step - loss: 37.7945 - val_loss: 91.3362
Epoch 83/1200
1/1 [==============================] - 0s 405ms/step - loss: 35.4569 - val_loss: 91.1832
Epoch 84/1200
1/1 [==============================] - 0s 393ms/step - loss

Epoch 153/1200
1/1 [==============================] - 0s 394ms/step - loss: 30.9757 - val_loss: 75.5985
Epoch 154/1200
1/1 [==============================] - 0s 409ms/step - loss: 27.4325 - val_loss: 81.3827
Epoch 155/1200
1/1 [==============================] - 0s 388ms/step - loss: 26.8568 - val_loss: 79.7035
Epoch 156/1200
1/1 [==============================] - 0s 386ms/step - loss: 29.8708 - val_loss: 76.3219
Epoch 157/1200
1/1 [==============================] - 0s 425ms/step - loss: 26.8639 - val_loss: 78.4541
Epoch 158/1200
1/1 [==============================] - 0s 394ms/step - loss: 39.5735 - val_loss: 85.3008
Epoch 159/1200
1/1 [==============================] - 0s 405ms/step - loss: 52.1363 - val_loss: 76.1846
Epoch 160/1200
1/1 [==============================] - 0s 400ms/step - loss: 34.3716 - val_loss: 78.4444
Epoch 161/1200
1/1 [==============================] - 0s 389ms/step - loss: 31.4399 - val_loss: 75.8232
Epoch 162/1200
1/1 [==============================] - 0s 419ms/s

1/1 [==============================] - 0s 400ms/step - loss: 25.1995 - val_loss: 57.9504
Epoch 231/1200
1/1 [==============================] - 0s 398ms/step - loss: 26.3250 - val_loss: 54.9945
Epoch 232/1200
1/1 [==============================] - 0s 385ms/step - loss: 26.9946 - val_loss: 56.1473
Epoch 233/1200
1/1 [==============================] - 0s 384ms/step - loss: 28.3315 - val_loss: 60.6331
Epoch 234/1200
1/1 [==============================] - 0s 484ms/step - loss: 22.6394 - val_loss: 52.7007
Epoch 235/1200
1/1 [==============================] - 0s 441ms/step - loss: 21.7352 - val_loss: 56.1246
Epoch 236/1200
1/1 [==============================] - 0s 414ms/step - loss: 19.3523 - val_loss: 50.1631
Epoch 237/1200
1/1 [==============================] - 0s 414ms/step - loss: 39.6824 - val_loss: 61.0105
Epoch 238/1200
1/1 [==============================] - 0s 489ms/step - loss: 19.6125 - val_loss: 58.1193
Epoch 239/1200
1/1 [==============================] - 0s 421ms/step - loss: 30.

1/1 [==============================] - 0s 442ms/step - loss: 17.0908 - val_loss: 30.4058
Epoch 308/1200
1/1 [==============================] - 0s 478ms/step - loss: 15.3824 - val_loss: 32.6861
Epoch 309/1200
1/1 [==============================] - 0s 480ms/step - loss: 17.9256 - val_loss: 31.7424
Epoch 310/1200
1/1 [==============================] - 0s 462ms/step - loss: 19.8163 - val_loss: 31.3995
Epoch 311/1200
1/1 [==============================] - 0s 444ms/step - loss: 21.1496 - val_loss: 30.1941
Epoch 312/1200
1/1 [==============================] - 0s 418ms/step - loss: 23.7531 - val_loss: 33.4383
Epoch 313/1200
1/1 [==============================] - 0s 398ms/step - loss: 25.9757 - val_loss: 29.0469
Epoch 314/1200
1/1 [==============================] - 0s 426ms/step - loss: 22.0148 - val_loss: 29.3231
Epoch 315/1200
1/1 [==============================] - 0s 414ms/step - loss: 15.3475 - val_loss: 30.1280
Epoch 316/1200
1/1 [==============================] - 0s 388ms/step - loss: 15.

...

1/1 [==============================] - 0s 397ms/step - loss: 4.6907 - val_loss: 3.1363
Epoch 859/1200
1/1 [==============================] - 0s 427ms/step - loss: 6.2789 - val_loss: 2.9188
Epoch 860/1200
1/1 [==============================] - 0s 471ms/step - loss: 7.2531 - val_loss: 3.1321
Epoch 861/1200
1/1 [==============================] - 0s 420ms/step - loss: 5.8586 - val_loss: 3.1279
Epoch 862/1200
1/1 [==============================] - 0s 409ms/step - loss: 5.0421 - val_loss: 2.8551
Epoch 863/1200
1/1 [==============================] - 0s 407ms/step - loss: 7.7185 - val_loss: 3.4826
Epoch 864/1200
1/1 [==============================] - 0s 430ms/step - loss: 14.5940 - val_loss: 2.7747
Epoch 865/1200
1/1 [==============================] - 0s 430ms/step - loss: 3.9387 - val_loss: 3.1155
Epoch 866/1200
1/1 [==============================] - 0s 418ms/step - loss: 3.8628 - val_loss: 4.0796
Epoch 867/1200
1/1 [==============================] - 0s 408ms/step - loss: 5.1628 - val_loss: 3

1/1 [==============================] - 0s 413ms/step - loss: 3.3091 - val_loss: 2.1982
Epoch 938/1200
1/1 [==============================] - 0s 410ms/step - loss: 4.1180 - val_loss: 3.2579
Epoch 939/1200
1/1 [==============================] - 0s 442ms/step - loss: 4.7892 - val_loss: 2.3989
Epoch 940/1200
1/1 [==============================] - 0s 459ms/step - loss: 4.1988 - val_loss: 2.6903
Epoch 941/1200
1/1 [==============================] - 0s 421ms/step - loss: 11.1235 - val_loss: 3.3548
Epoch 942/1200
1/1 [==============================] - 0s 468ms/step - loss: 6.7820 - val_loss: 1.8384
Epoch 943/1200
1/1 [==============================] - 0s 414ms/step - loss: 15.5618 - val_loss: 3.5274
Epoch 944/1200
1/1 [==============================] - 0s 470ms/step - loss: 4.8494 - val_loss: 2.4941
Epoch 945/1200
1/1 [==============================] - 0s 458ms/step - loss: 6.5805 - val_loss: 2.6747
Epoch 946/1200
1/1 [==============================] - 0s 430ms/step - loss: 4.2701 - val_loss: 

1/1 [==============================] - 0s 404ms/step - loss: 6.4998 - val_loss: 2.8550
Epoch 1017/1200
1/1 [==============================] - 0s 408ms/step - loss: 2.8307 - val_loss: 2.6288
Epoch 1018/1200
1/1 [==============================] - 0s 417ms/step - loss: 2.8793 - val_loss: 2.0696
Epoch 1019/1200
1/1 [==============================] - 0s 409ms/step - loss: 3.2782 - val_loss: 2.6140
Epoch 1020/1200
1/1 [==============================] - 0s 405ms/step - loss: 3.6456 - val_loss: 2.3491
Epoch 1021/1200
1/1 [==============================] - 0s 466ms/step - loss: 17.4330 - val_loss: 1.8071
Epoch 1022/1200
1/1 [==============================] - 0s 493ms/step - loss: 40.1395 - val_loss: 2.4566
Epoch 1023/1200
1/1 [==============================] - 0s 427ms/step - loss: 5.1700 - val_loss: 1.6054
Epoch 1024/1200
1/1 [==============================] - 0s 425ms/step - loss: 6.2406 - val_loss: 2.9558
Epoch 1025/1200
1/1 [==============================] - 0s 438ms/step - loss: 8.9317 - v

1/1 [==============================] - 1s 591ms/step - loss: 5.0063 - val_loss: 2.1108
Epoch 1096/1200
1/1 [==============================] - 1s 566ms/step - loss: 3.1724 - val_loss: 1.5786
Epoch 1097/1200
1/1 [==============================] - 1s 608ms/step - loss: 11.2451 - val_loss: 1.9513
Epoch 1098/1200
1/1 [==============================] - 1s 586ms/step - loss: 4.2866 - val_loss: 2.2708
Epoch 1099/1200
1/1 [==============================] - 0s 465ms/step - loss: 3.2903 - val_loss: 2.7688
Epoch 1100/1200
1/1 [==============================] - ETA: 0s - loss: 3.0436
Epoch 1100: saving model to LSTM+BN5--2022_12_19_14_47_37.hdf5
1/1 [==============================] - 1s 830ms/step - loss: 3.0436 - val_loss: 2.1070
Epoch 1101/1200
1/1 [==============================] - 1s 530ms/step - loss: 3.7183 - val_loss: 2.7597
Epoch 1102/1200
1/1 [==============================] - 1s 538ms/step - loss: 4.7829 - val_loss: 1.6777
Epoch 1103/1200
1/1 [==============================] - 1s 605ms/st

1/1 [==============================] - 1s 598ms/step - loss: 3.4417 - val_loss: 1.9578
Epoch 1174/1200
1/1 [==============================] - 1s 603ms/step - loss: 4.6626 - val_loss: 1.5721
Epoch 1175/1200
1/1 [==============================] - 1s 843ms/step - loss: 3.5563 - val_loss: 1.9517
Epoch 1176/1200
1/1 [==============================] - 1s 820ms/step - loss: 3.7336 - val_loss: 1.9502
Epoch 1177/1200
1/1 [==============================] - 1s 855ms/step - loss: 6.5818 - val_loss: 1.9312
Epoch 1178/1200
1/1 [==============================] - 1s 783ms/step - loss: 3.8917 - val_loss: 2.1947
Epoch 1179/1200
1/1 [==============================] - 1s 614ms/step - loss: 3.5496 - val_loss: 2.0196
Epoch 1180/1200
1/1 [==============================] - 1s 605ms/step - loss: 2.6328 - val_loss: 1.9289
Epoch 1181/1200
1/1 [==============================] - 1s 611ms/step - loss: 2.3080 - val_loss: 2.5143
Epoch 1182/1200
1/1 [==============================] - 1s 597ms/step - loss: 2.8765 - val